In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt


#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#Transforms
transformer=transforms.Compose([
    transforms.Resize((300,300)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    torchvision.transforms.Grayscale(num_output_channels=1)
])

#dataloader
train_path =  r"C:\Datasets\Train_set"
test_path = r"C:\Datasets\Test_set"
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

cuda
['stage1', 'stage2', 'stage3', 'stage4', 'stage5', 'stage6', 'stage7', 'stage8', 'stage9']


In [2]:
import torch.nn as nn

class MobileNetV1(nn.Module):
    def __init__(self, input_channel, n_classes):
        super().__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
                )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                # dw
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                # pw
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
                )

        self.model = nn.Sequential(
            conv_bn(input_channel, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7)
        )
        self.fc = nn.Linear(1024, n_classes)

        self.model.apply(self.init_weights)
        self.fc.apply(self.init_weights)

    def init_weights(self, layer):
        if type(layer) == nn.Conv2d:
            nn.init.kaiming_normal_(layer.weight, mode='fan_out')
        if type(layer) == nn.Linear:
            nn.init.normal_(layer.weight, std=1e-3)
        if type(layer) == nn.BatchNorm2d:
            nn.init.constant_(layer.weight, 1)
            nn.init.constant_(layer.bias, 0)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x
            

In [3]:
model=MobileNetV1(input_channel=1, n_classes=9).to(device)
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

77626 23645


In [4]:
import torch
import shutil
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)

In [5]:
y_pred = []
y_true = []
train_losses = []
test_accuracies = []
best_accuracy= 0.0
test_accuracy=0.0
train_accuracy=0.0
train_loss=0.0

In [6]:
def train_test(start_epochs, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred):
    #Model training and saving best model

    for epoch in range(start_epochs, num_epochs+1):

        #Evaluation and training on training dataset
        model.train()

        for i, (images,labels) in enumerate(train_loader):
            if torch.cuda.is_available():
                images=Variable(images.cuda())
                labels=Variable(labels.cuda())

            optimizer.zero_grad()

            outputs=model(images)
            loss=loss_function(outputs,labels)
            loss.backward()
            optimizer.step()

            train_loss+= loss.cpu().data*images.size(0)
            _,prediction=torch.max(outputs.data,1)

            train_accuracy+=int(torch.sum(prediction==labels.data))

        train_accuracy=train_accuracy/train_count
        train_loss=train_loss/train_count

        # Evaluation on testing dataset
        model.eval()

        for i, (images,labels) in enumerate(test_loader):
            if torch.cuda.is_available():
                images=Variable(images.cuda())
                labels=Variable(labels.cuda())

            outputs=model(images)
            _,prediction=torch.max(outputs.data,1)
            test_accuracy+=int(torch.sum(prediction==labels.data))

            output = (torch.max(torch.exp(outputs), 1)[1]).data.cpu().numpy()
            y_pred.extend(output) # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth

        test_accuracy=test_accuracy/test_count

        print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
        
        train_losses.append(train_loss.item())
        test_accuracies.append(test_accuracy)
        checkpoint = {
                'start_epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_accuracy': best_accuracy,
                'train_accuracy': train_accuracy,
                'test_accuracy': test_accuracy,
                'train_loss': train_loss,
                'train_losses':train_losses,
                'test_accuracies': test_accuracies,
                'y_true': y_true,
                'y_pred': y_pred
        }
        save_ckp(checkpoint, False, checkpoint_path='current_checkpoint_mobilenetv1_stagewise.pt', best_model_path='best_checkpoint_mobilenetv1_stagewise.model')
        #Save the best model
        if test_accuracy>best_accuracy:
            torch.save(model.state_dict(),'best_checkpoint_mobilenetv1_stagewise.model')
            best_accuracy=test_accuracy
            checkpoint['best_accuracy']=best_accuracy
            save_ckp(checkpoint, True, checkpoint_path='current_checkpoint_mobilenetv1_stagewise.pt', best_model_path='best_checkpoint_mobilenetv1_stagewise.model')

In [7]:
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_accuracy = checkpoint['best_accuracy']
    train_accuracy = checkpoint['train_accuracy']
    test_accuracy = checkpoint['test_accuracy']
    train_loss = checkpoint['train_loss']    
    train_losses = checkpoint['train_losses']    
    test_accuracies = checkpoint['test_accuracies']   
    start_epoch = checkpoint['start_epoch']
    y_true = checkpoint['y_true']
    y_pred = checkpoint['y_pred']
    return model, optimizer,start_epoch,best_accuracy, train_accuracy,test_accuracy,train_loss.item(),train_losses,test_accuracies,y_true,y_pred

In [8]:
def conf_matrix(y_true,y_pred):
     # constant for classes
   # classes = ('L4', 'day1', 'day10', 'day11', 'day12', 'day13', 'day14', 'day15', 'day16', 'day17', 'day18', 'day19', 'day2', 'day20', 'day21', 'day3', 'day4', 'day5', 'day6', 'day7', 'day8', 'day9', 'others')

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix), index = [i for i in classes],
                         columns = [i for i in classes])
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('output.png')

In [9]:
def learn_curve_train_losses(train_losses):
    plt.plot(train_losses,'-o')
    plt.xlabel('epoch')
    plt.ylabel('losses')
    plt.legend(['Train'])
    plt.title('Train Losses Per Epoch')

    plt.show()
    
def learn_curve_test_accuracies(test_accuracies):
    plt.plot(test_accuracies,'-r')
    plt.xlabel('epoch')
    plt.ylabel('test_accuracy')
    plt.legend(['Test Accuracy'])
    plt.title('Test Accuracy Per Epoch')
    plt.show()


In [ ]:
num_epochs = 40

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 1 Train Loss: tensor(1.3857) Train Accuracy: 0.4393373354288512 Test Accuracy: 0.5083104250370057


In [ ]:
num_epochs = 40

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 2 Train Loss: tensor(0.8175) Train Accuracy: 0.6873269180085979 Test Accuracy: 0.642948120550858
Epoch: 3 Train Loss: tensor(0.5852) Train Accuracy: 0.7797862485110402 Test Accuracy: 0.7032625480279361
Epoch: 4 Train Loss: tensor(0.4868) Train Accuracy: 0.8191685747848467 Test Accuracy: 0.6816537645399885
Epoch: 5 Train Loss: tensor(0.4172) Train Accuracy: 0.846891752358421 Test Accuracy: 0.6845710151729557
Epoch: 6 Train Loss: tensor(0.3703) Train Accuracy: 0.8648242456361575 Test Accuracy: 0.768859571622549
Epoch: 7 Train Loss: tensor(0.3306) Train Accuracy: 0.8811978567006626 Test Accuracy: 0.7748263421260996
Epoch: 8 Train Loss: tensor(0.3048) Train Accuracy: 0.8899451369110442 Test Accuracy: 0.7554144566014857
Epoch: 9 Train Loss: tensor(0.2794) Train Accuracy: 0.8993750798074989 Test Accuracy: 0.7446714068283613
Epoch: 10 Train Loss: tensor(0.2613) Train Accuracy: 0.9058807535500968 Test Accuracy: 0.763490999002192
Epoch: 11 Train Loss: tensor(0.2452) Train Accuracy: 0.913

In [ ]:
num_epochs = 40

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 26 Train Loss: tensor(0.1352) Train Accuracy: 0.9520515357646798 Test Accuracy: 0.8316268249111702
Epoch: 27 Train Loss: tensor(0.1314) Train Accuracy: 0.9535716390324862 Test Accuracy: 0.8168674826316308
Epoch: 28 Train Loss: tensor(0.1281) Train Accuracy: 0.9554653540262158 Test Accuracy: 0.8205885754909127
Epoch: 29 Train Loss: tensor(0.1275) Train Accuracy: 0.955607083520393 Test Accuracy: 0.8200389337523997
Epoch: 30 Train Loss: tensor(0.1208) Train Accuracy: 0.9581835416881396 Test Accuracy: 0.813483613403838
Epoch: 31 Train Loss: tensor(0.1178) Train Accuracy: 0.958209339442219 Test Accuracy: 0.8212228159701165
Epoch: 32 Train Loss: tensor(0.1215) Train Accuracy: 0.9561868215461242 Test Accuracy: 0.8306543126587426
Epoch: 33 Train Loss: tensor(0.1102) Train Accuracy: 0.9603477724837237 Test Accuracy: 0.8057445825465283
Epoch: 34 Train Loss: tensor(0.1127) Train Accuracy: 0.9604895311850732 Test Accuracy: 0.837631877546312
Epoch: 35 Train Loss: tensor(0.1110) Train Accurac

In [ ]:
num_epochs = 50

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 39 Train Loss: tensor(0.0961) Train Accuracy: 0.9659645415713843 Test Accuracy: 0.8438504232363405
Epoch: 40 Train Loss: tensor(0.1010) Train Accuracy: 0.9648953438866046 Test Accuracy: 0.838775379590748
Epoch: 41 Train Loss: tensor(0.0952) Train Accuracy: 0.9674975510182656 Test Accuracy: 0.853281403061095
Epoch: 42 Train Loss: tensor(0.0959) Train Accuracy: 0.9670080578356611 Test Accuracy: 0.8522247105689601
Epoch: 43 Train Loss: tensor(0.0947) Train Accuracy: 0.9672656971640666 Test Accuracy: 0.839114071673105
Epoch: 44 Train Loss: tensor(0.0947) Train Accuracy: 0.9670080548488543 Test Accuracy: 0.8370411974654969
Epoch: 45 Train Loss: tensor(0.0936) Train Accuracy: 0.9671368743469309 Test Accuracy: 0.8309087350897638


In [21]:
num_epochs = 50

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 46 Train Loss: tensor(0.0884) Train Accuracy: 0.9690305714177511 Test Accuracy: 0.8415661200564639
Epoch: 47 Train Loss: tensor(0.0901) Train Accuracy: 0.9689533021226319 Test Accuracy: 0.8554807175352107
Epoch: 48 Train Loss: tensor(0.0878) Train Accuracy: 0.970177117889652 Test Accuracy: 0.8474457805336238
Epoch: 49 Train Loss: tensor(0.0880) Train Accuracy: 0.9696876069502214 Test Accuracy: 0.8643623364677748
Epoch: 50 Train Loss: tensor(0.0842) Train Accuracy: 0.9704090084199488 Test Accuracy: 0.8581883849581928


In [22]:
num_epochs = 55

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 51 Train Loss: tensor(0.0827) Train Accuracy: 0.971323659714637 Test Accuracy: 0.8577229092148428
Epoch: 52 Train Loss: tensor(0.0834) Train Accuracy: 0.9710016144546894 Test Accuracy: 0.8461771081797088
Epoch: 53 Train Loss: tensor(0.0805) Train Accuracy: 0.9723928967306631 Test Accuracy: 0.8704100730432726
Epoch: 54 Train Loss: tensor(0.0784) Train Accuracy: 0.9732560275281057 Test Accuracy: 0.8578503028155231
Epoch: 55 Train Loss: tensor(0.0809) Train Accuracy: 0.9719935750396456 Test Accuracy: 0.85666558893224


In [23]:
num_epochs = 75

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 56 Train Loss: tensor(0.0784) Train Accuracy: 0.9732302578205118 Test Accuracy: 0.857892013769885
Epoch: 57 Train Loss: tensor(0.0772) Train Accuracy: 0.9731787446249687 Test Accuracy: 0.8603027232824602
Epoch: 58 Train Loss: tensor(0.0753) Train Accuracy: 0.9738357403285577 Test Accuracy: 0.86825376623909
Epoch: 59 Train Loss: tensor(0.0760) Train Accuracy: 0.9736425145665154 Test Accuracy: 0.8594150244773203
Epoch: 60 Train Loss: tensor(0.0755) Train Accuracy: 0.9737326880492949 Test Accuracy: 0.8542549974635008
Epoch: 61 Train Loss: tensor(0.0743) Train Accuracy: 0.9740289816902592 Test Accuracy: 0.8729902412771183
Epoch: 62 Train Loss: tensor(0.0727) Train Accuracy: 0.974582923620716 Test Accuracy: 0.8706226682275863
Epoch: 63 Train Loss: tensor(0.0729) Train Accuracy: 0.9746215776018811 Test Accuracy: 0.8649554080214941
Epoch: 64 Train Loss: tensor(0.0730) Train Accuracy: 0.9745829312547033 Test Accuracy: 0.8654626752128578
Epoch: 65 Train Loss: tensor(0.0693) Train Accurac

In [10]:
num_epochs = 105

import os.path

if os.path.isfile("current_checkpoint_mobilenetv1_stagewise.pt"):
    ckp_path = "current_checkpoint_mobilenetv1_stagewise.pt"
    model, optimizer, start_epoch, best_accuracy, train_accuracy,test_accuracy,train_loss,train_losses,test_accuracies,y_true,y_pred  = load_ckp(ckp_path, model, optimizer)
    train_test(start_epoch, best_accuracy,train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)
else:
    train_test(1,best_accuracy, train_accuracy,test_accuracy,train_loss, train_losses, test_accuracies,y_true,y_pred)


Epoch: 76 Train Loss: tensor(0.0648) Train Accuracy: 0.977842180888708 Test Accuracy: 0.8819148446249595
Epoch: 77 Train Loss: tensor(0.0642) Train Accuracy: 0.9777262494805978 Test Accuracy: 0.8735412101858586
Epoch: 78 Train Loss: tensor(0.0635) Train Accuracy: 0.9788985356227229 Test Accuracy: 0.8703266458536767
Epoch: 79 Train Loss: tensor(0.0609) Train Accuracy: 0.9787568456256361 Test Accuracy: 0.8778968207505119
Epoch: 80 Train Loss: tensor(0.0593) Train Accuracy: 0.9795555452663492 Test Accuracy: 0.8875820637268239
Epoch: 81 Train Loss: tensor(0.0642) Train Accuracy: 0.9783446210747077 Test Accuracy: 0.8841990941875122
Epoch: 82 Train Loss: tensor(0.0613) Train Accuracy: 0.9789114258704696 Test Accuracy: 0.8762903023512026
Epoch: 83 Train Loss: tensor(0.0597) Train Accuracy: 0.9797101346382124 Test Accuracy: 0.866562752814648
Epoch: 84 Train Loss: tensor(0.0593) Train Accuracy: 0.9799162614347594 Test Accuracy: 0.8841559129944095
Epoch: 85 Train Loss: tensor(0.0593) Train Accur